In [1]:
import pandas as pd
import os

## US Census Bureau Data

In [2]:
DATA_FOLDER = '../data/ACSST/'

In [3]:
# Read Data
df_metadata = pd.read_csv('../data/ACSST/ACSST1Y2010.S1301-Column-Metadata.csv')
df_metadata.loc[len(df_metadata)] = ['YEAR', 'Year of Data Collection']

# Get a mapping of column codes and their description
# This is done to avoid a complicated multi-index dataframe

header_mapper = dict(zip(df_metadata['Column Name'], df_metadata['Label']))

In [4]:
# Extract and combine all CSV files
list_df = []
for file in os.listdir(DATA_FOLDER):
    if file.split('.')[0].startswith('ACSST1Y'):
        if file.split('-')[-1] == 'Data.csv':
            df_temp = pd.read_csv(os.path.join(DATA_FOLDER, file), header=[0], skiprows=[1])
            df_temp['YEAR'] = [int(file.split('.')[0][-4:])]
            df_temp = df_temp.drop('Unnamed: 642', axis=1)
            list_df.append(df_temp)
df_fertility = pd.concat(list_df)

# Drop columns with all nulls
df_fertility = df_fertility.dropna(axis=1, how='all')

In [5]:
# Subset metadata to focus on columns where data exists
header_mapper = {key: header_mapper[key] for key in df_fertility.columns}

# Example
header_mapper['S1301_C04_001E']

'Women with births in the past 12 months!!Rate per 1,000 women!!Estimate!!Women 15 to 50 years'

In [6]:
# After carefully going through all the columns, these seem to be of interest
columns_of_interest = ['S1301_C04_001E', 'S1301_C04_001M', 'YEAR']
df_fertility = df_fertility[columns_of_interest]

## Centers for Disease Control and Prevention Data

In [7]:
# Load Premature Death data
df_premature_mortality = pd.read_csv('../data/CDC20N2U004025.csv')

# Centers for Disease Control and Prevention, Premature Death Rate for Yavapai County, AZ [CDC20N2U004025], retrieved from FRED, 
# Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/CDC20N2U004025, November 15, 2023.

In [8]:
# Add Premature Death Rate to the data dictionary
header_mapper['CDC20N2U004025'] = "The crude death rate is the number of deaths reported each calendar year divided by the population, multiplied by 100,000. Premature death rate includes all deaths where the deceased is younger than 75 years of age. 75 years of age is the standard consideration of a premature death according to the CDC's definition of Years of Potential Life Loss."

In [9]:
# Extract year of date and convert to int
df_premature_mortality.DATE = df_premature_mortality.DATE.str[:4].astype(int)
df_premature_mortality = df_premature_mortality.rename(columns={'DATE': 'YEAR'})

In [10]:
df_main = pd.merge(df_premature_mortality, df_fertility, on='YEAR', how='outer')

# IHME Data

In [11]:
df = pd.read_csv('../data/IHME-GBD_2019_DATA-f69f76a7-1/IHME-GBD_2019_DATA-f69f76a7-1.csv')

In [12]:
# Choose only rates to get per-capita numbers
df = df[df.metric_name == 'Rate']

# Choose only Incidence and Death Rates
df = df[df.measure_name != 'Prevalence']

In [13]:
df = df[['measure_name', 'year', 'val', 'upper', 'lower']]

In [14]:
# Merge Asthma incidnece and death data on year
df = pd.merge(df[df['measure_name'] == 'Incidence'], df[df['measure_name'] == 'Deaths'], on='year', suffixes=('_incidence', '_deaths')).drop(['measure_name_incidence', 'measure_name_deaths'], axis=1)
df = df[['val_deaths', 'val_incidence', 'year']]
df = df.rename(columns={'year': 'YEAR'})

# Merge

In [15]:
# Merge all dataframes on year
df_main = pd.merge(df, df_main, on='YEAR', how='outer')

In [16]:
df_main.head()

,val_deaths,val_incidence,YEAR,CDC20N2U004025,S1301_C04_001E,S1301_C04_001M
0,2.223420,1404.898804,1990,NaN,NaN,NaN
1,2.251678,1328.627715,1991,NaN,NaN,NaN
2,2.225029,1258.860831,1992,NaN,NaN,NaN
3,2.341241,1197.121010,1993,NaN,NaN,NaN
4,2.320625,1146.080036,1994,NaN,NaN,NaN


In [17]:
# Save Dataframe
df_main.to_csv('../data/merged_data.csv')

In [19]:
# Save metadata
header_mapper['val_deaths'] = 'Rate of deaths caused due to Asthma'
header_mapper['val_incidence'] = 'Rate of Asthma cases reported'

In [18]:
import json
with open('../data/header_mapper.json', 'w') as f:
    json.dump(header_mapper, f)

In [20]:
header_mapper

{'GEO_ID': 'Geography',
 'NAME': 'Geographic Area Name',
 'S1301_C01_001E': 'Total!!Estimate!!Women 15 to 50 years',
 'S1301_C01_001M': 'Total!!Margin of Error!!Women 15 to 50 years',
 'S1301_C01_002E': 'Total!!Estimate!!15 to 19 years',
 'S1301_C01_002M': 'Total!!Margin of Error!!15 to 19 years',
 'S1301_C01_003E': 'Total!!Estimate!!20 to 34 years',
 'S1301_C01_003M': 'Total!!Margin of Error!!20 to 34 years',
 'S1301_C01_004E': 'Total!!Estimate!!35 to 50 years',
 'S1301_C01_004M': 'Total!!Margin of Error!!35 to 50 years',
 'S1301_C01_005E': 'Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race',
 'S1301_C01_005M': 'Total!!Margin of Error!!RACE AND HISPANIC OR LATINO ORIGIN!!One race',
 'S1301_C01_005MA': 'Annotation of Total!!Margin of Error!!RACE AND HISPANIC OR LATINO ORIGIN!!One race',
 'S1301_C01_005EA': 'Annotation of Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race',
 'S1301_C01_006E': 'Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White',
